## About

In this notebook we prepare a simple solution for the [kaggle challenge on higgs.](https://inclass.kaggle.com/c/mlhep-2016-higgs-detection)

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

import pandas
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
import root_numpy
data = pandas.DataFrame(root_numpy.root2array('datasets/public_train_100000.root'))
#data = data[:50000]
test = pandas.DataFrame(root_numpy.root2array('datasets/public_test.root'))

In [4]:
features = list(set(data.columns) - {'event_id', 'target'})
#features

### Prepare high-level features for training

In [72]:
#high_level_features = ['m_jj', 'm_jjj', 'm_jlv', 'm_wwbb', 'm_bb', 'm_wbb', 'm_lv']
#high_level_features_a = ['m_jj', 'm_jjj', 'm_jlv', 'm_wwbb', 'm_bb', 'm_wbb', 'm_lv', 'lepton_pt','mem_pt']
#high_level_features_b = ['m_jj', 'm_jjj', 'm_jlv', 'm_wwbb', 'm_bb', 'm_wbb', 'lepton_pt','mem_pt','jet1_pt','jet2_pt','jet3_pt','jet4_pt']
#high_level_features_c = ['m_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_wwbb', 'm_bb', 'm_wbb', 'lepton_pt','mem_pt','jet1_pt','jet2_pt','jet3_pt','jet4_pt']
#high_level_features_d = ['m_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_wwbb', 'm_bb', 'm_wbb', 'lepton_pt','mem_pt','jet1_pt','jet2_pt','jet3_pt','jet4_pt','j_HT','phi_bb']
#high_level_features_e = ['m_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_wwbb', 'm_bb', 'm_wbb', 'lepton_pt','mem_pt','jet1_pt','jet2_pt','jet3_pt','jet4_pt','j_HT','phi_bb','LT']
data['j_HT'] = data['jet1_pt'] + data['jet2_pt'] + data['jet3_pt'] + data['jet4_pt']
test['j_HT'] = test['jet1_pt'] + test['jet2_pt'] + test['jet3_pt'] + test['jet4_pt']
data['phi_bb'] = data['jet1_phi'] + data['jet2_phi'] + data['jet3_phi'] + data['jet4_phi']
test['phi_bb'] = test['jet1_phi'] + test['jet2_phi'] + test['jet3_phi'] + test['jet4_phi']
data['LT'] = data['lepton_pt'] + data['mem_pt']
test['LT'] = test['lepton_pt'] + test['mem_pt']
high_level_features = ['m_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_wwbb', 'm_bb', 'm_wbb', 'jet1_pt','jet2_pt','jet3_pt','jet4_pt','j_HT','phi_bb','lepton_pt','mem_pt','LT']#

### Divide training data into 2 parts 
`train_test_split` function is used to divide into 2 parts to preserve quality overestimating.

In [73]:
training_data, validation_data = train_test_split(data, random_state=11, train_size=0.66)

### XGBoost

In [74]:
from rep.estimators import XGBoostClassifier
from sklearn.cross_validation import cross_val_score

xgboost_cv = cross_val_score(XGBoostClassifier(max_depth = 6, min_child_weight = 1, gamma=0.3, colsample=1, nthreads=10),
                             training_data[high_level_features].astype(np.float64),
                             training_data.target.astype(np.int64), cv=4, scoring="roc_auc")
print(xgboost_cv.mean(), xgboost_cv.std())

#0.7668823025 senza opzioni con lepton_pt, mem_phi 100'000 eventi 
#0.72716605849516347 senza opzioni con lepton_pt, mem_phi 10'000 eventi
#high level features a:
#max_depth 5: 0.77373192433167481
#max_depth 4: 0.7748102315375196, gamma=0.3: 0.7750851890287791
#max_depth 6: 0.77178583194465911
# high level features b:
#max_depth 6, gamma=0.3 0.78655990568943845
# high level features c:
#max_depth 6, gamma=0.3 0.78747491255687363
# high level features c with Ht and pt:
#max_depth 6, gamma=0.3 0.78959440814072357
# high level features c with Ht only:
#max_depth 6, gamma=0.3 0.78882205872533173
# high level features d:
#max_depth 6, gamma=0.3 0.78911150552696641
# high level features e:
#max_depth 6, gamma=0.3 0.78987168613515957

(0.78987067160674362, 0.0017453148557578547)


In [8]:
len(training_data)

66000

In [63]:
#from rep.estimators import XGBoostClassifier
#from sklearn.cross_validation import cross_val_score

#xgboost_cv = cross_val_score(XGBoostClassifier(max_depth = 5, min_child_weight = 1, gamma=0),
#                             training_data[high_level_features].astype(np.float64),
#                             training_data.target.astype(np.int64), cv=4, scoring="roc_auc")
#print(xgboost_cv.mean(), xgboost_cv.std())


#0.7668823025 senza opzioni con lepton_pt, mem_phi 100'000 eventi 
#0.72716605849516347 senza opzioni con lepton_pt, mem_phi 10'000 eventi
from sklearn.ensemble import BaggingClassifier
xgboost_bag_cv = cross_val_score(BaggingClassifier(base_estimator=XGBoostClassifier(max_depth = 6, min_child_weight = 1, gamma=0., colsample=1, nthreads=10),n_estimators=10),
                             training_data[high_level_features].astype(np.float64),
                             training_data.target.astype(np.int64), cv=4, scoring="roc_auc")
print(xgboost_bag_cv.mean(), xgboost_bag_cv.std())
#boost_bagg = BaggingClassifier(base_estimator=XGBoostClassifier(max_depth = 5, min_child_weight = 1, gamma=0),n_estimators=34, n_jobs=10)
#boost_bagg.fit(training_data[high_level_features].astype(np.float64), training_data.target.astype(np.int64))

#high level features a
###0.78144373296283975 con max_depth = 5
###0.77968141225306831 con max_depth = 4
###0.7823899079878065 con max_depth = 6, gamma=0.3 0.78169084626057739
# high level features b:
#max_depth 6, gamma=0.3 0.7972091448484645
#max_depth 6, gamma=0 0.79791828545435561
# high level features c:
#max_depth 6, gamma=0. 0.7992048882561511
# high level features c with Ht an pt
#max_depth 6, gamma=0.0.80147524606003273
# high level features c with Ht only
#max_depth 6, gamma=0.  0.79897399081169285
#high level features d:
#max_depth 6, gamma=0 0.80011445909587442
#high level features d:
#max_depth 6, gamma=0 0.80165832047956576

(0.80165832047956576, 0.002120279817989415)


In [64]:
from rep.estimators import XGBoostClassifier
#from sklearn.cross_validation import cross_val_score

#xgboost_cv = cross_val_score(XGBoostClassifier(max_depth = 5, min_child_weight = 1, gamma=0),
#                             training_data[high_level_features].astype(np.float64),
#                             training_data.target.astype(np.int64), cv=4, scoring="roc_auc")
#print(xgboost_cv.mean(), xgboost_cv.std())


#0.7668823025 senza opzioni con lepton_pt, mem_phi 100'000 eventi 
#0.72716605849516347 senza opzioni con lepton_pt, mem_phi 10'000 eventi
from sklearn.ensemble import BaggingClassifier
#xgboost_bag_cv = cross_val_score(BaggingClassifier(base_estimator=XGBoostClassifier(max_depth = 5, min_child_weight = 1, gamma=0),n_estimators=34),
#                             training_data[high_level_features].astype(np.float64),
#                             training_data.target.astype(np.int64), cv=4, scoring="roc_auc")
#print(xgboost_bag_cv.mean(), xgboost_bag_cv.std())
boost_bagg = BaggingClassifier(base_estimator=XGBoostClassifier(max_depth = 6, min_child_weight = 1, gamma=0, 
                                                                colsample=1, n_estimators=10, nthreads=10))
boost_bagg.fit(training_data[high_level_features].astype(np.float64), training_data.target.astype(np.int64))

BaggingClassifier(base_estimator=XGBoostClassifier(base_score=0.5, colsample=1, eta=0.3, features=None,
         gamma=0, max_depth=6, min_child_weight=1, missing=-999.0,
         n_estimators=10, nthreads=10, num_feature=None, random_state=0,
         scale_pos_weight=1.0, subsample=1.0, verbose=0),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [ ]:
#Risultati:
#xgboost bag 25 n_estimator: 748619196
#xgboost bag 30 n_estimator: 74810733
#xgboost bag 35 n_estimator: 74842935567290336
#xgboost bag 34 n_estimator: 74889629284010684
#"                      , max depth 5: 0.74948804173465056
# " "                                , min child 1, gamma 0: 0.75022381545276473
# + colsample=0.8: 0.74879395566741258
# + colsample= 0.8 su 100'000: 0.77806206257685062
# colsample=1 su 100'000 0.77841176129880307
# max depths 6?
#se tolgo mjjj, 30 estimators: peggio scende

In [ ]:
from rep.estimators import XGBoostClassifier
xg = XGBoostClassifier(max_depth = 5, min_child_weight = 1, gamma=0, colsample=1)
xg.fit(training_data[high_level_features].astype(np.float64), training_data.target.astype(np.int64))

### Ada on XGBoost

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
ada_on_xgb = cross_val_score(AdaBoostClassifier(base_estimator=XGBoostClassifier()),
                              training_data[high_level_features].astype(np.float64), training_data.target.astype(np.int64), cv=4, scoring="roc_auc")
print(ada_on_xgb.mean(), ada_on_xgb.std())

In [ ]:
xg_ada = AdaBoostClassifier(base_estimator=XGBoostClassifier())
xg_ada.fit(training_data[high_level_features].astype(np.float64), training_data.target.astype(np.int64))

In [ ]:
#0.71559484247046334 ada cross val: senza opzioni con lepton_pt, mem_phi 10'000 eventi

In [ ]:
# predict validation sample (probability for each event)
proba = xg.predict_proba(validation_data[high_level_features].astype(np.float64))
#proba_ada = xg_ada.predict_proba(validation_data[high_level_features].astype(np.float64))
#proba_boost_bag = boost_bagg.predict_proba(validation_data[high_level_features].astype(np.float64))

In [65]:
# predict validation sample (probability for each event)
#proba = xg.predict_proba(validation_data[high_level_features].astype(np.float64))
#proba_ada = xg_ada.predict_proba(validation_data[high_level_features].astype(np.float64))
proba_boost_bag = boost_bagg.predict_proba(validation_data[high_level_features].astype(np.float64))

In [ ]:
proba
proba_ada

### Compute quality (ROC AUC) on the validation set (to prevent overestimating quality)

In [ ]:
# take probability to be 1 class to compute ROC AUC
#
roc_auc_score(validation_data.target, proba[:, 1])
#0.73411680585983141 con xgb da solo, senza opzioni, lep_pt, mem_phi, 10'000 eventi


In [ ]:
# take probability to be 1 class to compute ROC AUC
#
roc_auc_score(validation_data.target, proba_ada[:, 1])
#0.7176807190099479 con xgb da solo, senza opzioni, lep_pt, mem_phi, 10'000 eventi


In [66]:
# take probability to be 1 class to compute ROC AUC
#
roc_auc_score(validation_data.target, proba_boost_bag[:, 1])
#0.7176807190099479 con xgb da solo, senza opzioni, lep_pt, mem_phi, 10'000 eventi


0.79228100410996594

## Prepare submission to kaggle

In [ ]:
kaggle_proba = boost_bagg.predict_proba(test[high_level_features].astype(np.float64))[:, 1]

In [67]:
# predict test sample
kaggle_proba = boost_bagg.predict_proba(test[high_level_features].astype(np.float64))[:, 1]
#kaggle_proba = xg.predict_proba(test[high_level_features].astype(np.float64))[:, 1]
kaggle_ids = test.event_id

In [68]:
from IPython.display import FileLink
def create_solution(ids, proba, filename='xg_bagg_tuned_setoffeatures_e.csv'):
    """saves predictions to file and provides a link for downloading """
    pandas.DataFrame({'event_id': ids, 'prediction': proba}).to_csv('datasets/{}'.format(filename), index=False)
    return FileLink('datasets/{}'.format(filename))
    
create_solution(kaggle_ids, kaggle_proba)

/notebooks/higgs_kaggle/datasets/xg_bagg_tuned_setoffeatures_e.csv

In [ ]:
!ls -lah datasets/